In [7]:
import numpy as np
from sklearn.cluster import KMeans
from stl import mesh  # You may need to install the 'numpy-stl' library for STL file manipulation
import math

class Teeth:
    def __init__(self, data, num_cluster):
        self.num_cluster = num_cluster
        self.data=data

    def cluster_teeth(self):
        # Perform clustering on the loaded data
        kmeans = KMeans(n_clusters=self.num_clusters, random_state=42)
        labels = kmeans.fit_predict(self.data)
        return labels
    def rotate_teeth(self, angle_degrees):
            # Convert angle to radians
            angle_radians = math.radians(angle_degrees)

            # Define rotation matrix for the Z-axis (assuming teeth are in the XY plane)
            rotation_matrix = np.array([
                [math.cos(angle_radians), -math.sin(angle_radians), 0],
                [math.sin(angle_radians), math.cos(angle_radians), 0],
                [0, 0, 1]
            ])

            # Apply rotation to each point in the teeth data
            self.data = np.dot(self.data, rotation_matrix)

In [13]:
class Jaw:

    def __init__(self, stl_files, num_teeth):
        self.stl_files = stl_files
        self.num_teeth=num_teeth
        self.teeth_list = self.load_teeth()
        self.dict_teeth = {
        'Central Incisor': 0,
        'Lateral Incisor': 1,
        'Canine (Cuspid)': 2,
        'First Premolar (Bicuspid)': 3,
        'Second Premolar (Bicuspid)': 4,
        'First Molar': 5,
        'Second Molar': 6,
        'Third Molar (Wisdom Tooth)': 7,
        'Upper Right Central Incisor': 8,
        'Upper Right Lateral Incisor': 9,
        'Upper Right Canine (Cuspid)': 10,
        'Upper Right First Premolar (Bicuspid)': 11,
        'Upper Right Second Premolar (Bicuspid)': 12,
        'Upper Right First Molar': 13,
        'Upper Right Second Molar': 14,
        'Upper Right Third Molar (Wisdom Tooth)': 15,
        'Upper Left Central Incisor': 16,
        'Upper Left Lateral Incisor': 17,
        'Upper Left Canine (Cuspid)': 18,
        'Upper Left First Premolar (Bicuspid)': 19,
        'Upper Left Second Premolar (Bicuspid)': 20,
        'Upper Left First Molar': 21,
        'Upper Left Second Molar': 22,
        'Upper Left Third Molar (Wisdom Tooth)': 23,
        'Lower Right Central Incisor': 24,
        'Lower Right Lateral Incisor': 25,
        'Lower Right Canine (Cuspid)': 26,
        'Lower Right First Premolar (Bicuspid)': 27,
        'Lower Right Second Premolar (Bicuspid)': 28,
        'Lower Right First Molar': 29,
        'Lower Right Second Molar': 30,
        'Lower Right Third Molar (Wisdom Tooth)': 31,
        'Lower Left Central Incisor': 32,
        'Lower Left Lateral Incisor': 33,
        'Lower Left Canine (Cuspid)': 34,
        'Lower Left First Premolar (Bicuspid)': 35,
        'Lower Left Second Premolar (Bicuspid)': 36,
        'Lower Left First Molar': 37,
        'Lower Left Second Molar': 38,
        'Lower Left Third Molar (Wisdom Tooth)': 39
    }
    def cluster_teeth(self,mesh_data):
        # Perform clustering on the loaded data
        kmeans = KMeans(n_clusters=self.num_teeth, random_state=42)
        labels = kmeans.fit_predict(mesh_data)
        return labels
    
    def load_teeth(self):
        teeth_list = []
        for stl_file in self.stl_files:
            mesh_data = mesh.Mesh.from_file(stl_file).vectors.reshape(-1, 3)
            self.labels=self.cluster_teeth(mesh_data)
            # Get data points for each label
            clustered_data = {}
            for label in range(self.num_teeth):
                points_in_cluster = mesh_data[self.labels == label]
                clustered_data[label] = points_in_cluster

            # Print or use the clustered data
            for label, points in clustered_data.items():
                teeth_instance = Teeth(points, label)
                teeth_list.append(teeth_instance)
        return teeth_list
    

    def print_summary(self):
        print("Jaw Summary:")
        print("-" * 40)
        print(f"Number of Teeth: {len(self.teeth_list)}")
        print()

        for i, teeth_instance in enumerate(self.teeth_list):
            print(f"Teeth {i + 1} Summary:")
            print(f"  Class Number: {teeth_instance.class_number}")
            print(f"  Data Shape: {teeth_instance.data.shape}")
            print()

    def align_teeth(self):
        # Find the lowest point for each teeth instance
        lowest_points = [np.min(teeth_instance.data[:, 2]) for teeth_instance in self.teeth_list]

        # Find the global minimum (lowest point among all teeth)
        global_min = np.min(lowest_points)

        # Adjust the position of each teeth to align with the global minimum
        for i, teeth_instance in enumerate(self.teeth_list):
            offset = global_min - lowest_points[i]
            teeth_instance.data[:, 2] += offset
            
    def compute_distances_and_depths(self):
        # Assuming specific indices for the canine, second premolar, and first molar
        # Replace these indices with the actual indices of your teeth clusters
        canine_indices = np.where(self.class_number == 0)[0]
        second_premolar_indices = np.where(self.class_number == 1)[0]
        first_molar_indices = np.where(self.class_number == 2)[0]

        # Compute inter-canine distance
        inter_canine_distance = np.linalg.norm(self.data[canine_indices[0]] - self.data[canine_indices[1]])

        # Compute inter-second-premolar distance
        inter_second_premolar_distance = np.linalg.norm(self.data[second_premolar_indices[0]] - self.data[second_premolar_indices[1]])

        # Compute inter-first-molar distance
        inter_first_molar_distance = np.linalg.norm(self.data[first_molar_indices[0]] - self.data[first_molar_indices[1]])

        # Compute depth of the canine
        depth_canine = np.max(self.data[canine_indices, 2]) - np.min(self.data[canine_indices, 2])

        # Compute depth of the second premolar
        depth_second_premolar = np.max(self.data[second_premolar_indices, 2]) - np.min(self.data[second_premolar_indices, 2])

        # Compute depth of the first molar
        depth_first_molar = np.max(self.data[first_molar_indices, 2]) - np.min(self.data[first_molar_indices, 2])

        return inter_canine_distance, inter_second_premolar_distance, inter_first_molar_distance, depth_canine, depth_second_premolar, depth_first_molar
    
    def align_teeth_clusters(self):
        # Find the centroid of each tooth cluster
        centroids = []
        for label in range(self.num_clusters):
            cluster_indices = np.where(self.class_number == label)[0]
            cluster_centroid = np.mean(self.data[cluster_indices], axis=0)
            centroids.append(cluster_centroid)

        # Sort the centroids based on their X-coordinate
        sorted_centroids = sorted(enumerate(centroids), key=lambda x: x[1][0])

        # Map the cluster numbers to the sorted order
        cluster_mapping = {label: sorted_order for sorted_order, (label, _) in enumerate(sorted_centroids)}

        # Update the class_number attribute based on the sorted order
        self.class_number = np.array([cluster_mapping[label] for label in self.class_number])


# Example Usage:
# Provide a list of paths to your STL files and the desired number of clusters
stl_files = [r"C:\Users\Emman\Desktop\JE\Ortho\STL-Segmentation\OrthoCAD_Export_43495989\43495989_shell_occlusion_u.stl"]
num_clusters = 30

# Create Jaw object
jaw_instance = Jaw(stl_files, num_clusters)
jaw_instance.print_summary()


# Access the Teeth instances within Jaw
for i, teeth_instance in enumerate(jaw_instance.teeth_list):
    print(f"Teeth {i + 1} - Class Number:", teeth_instance.class_number)
    print(f"Teeth {i + 1} - Data Shape:", teeth_instance.data.shape)
    print()


c:\Users\Emman\anaconda3\envs\ortho\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb0 in position 2: invalid start byte

In [5]:
jaw_instance.teeth_list